In [ ]:
# General Libs
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
%matplotlib inline

**Detecção de retinopatia diabética para evitar cegueira**

![](https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/APTOS%202019%20Blindness%20Detection/aux_img.png)


**Legend**
* 0 - No DR
* 1 - Mild
* 2 - Moderate
* 3 - Severe
* 4 - Proliferative DR

In [ ]:
def import_api_key(user, key):
    !touch kaggle.json
    !echo '{"username":"'+user+'","key":"'+key+'"}' > ./kaggle.json
    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    !ls -la ~/.kaggle/
    from kaggle.api.kaggle_api_extended import KaggleApi
    
import_api_key('eberthfelipe', '')    

# Load Dataset

In [ ]:
!ls -la ../input/aptos2019-blindness-detection/

In [ ]:
TRAINING_DIR = '../input/aptos2019-blindness-detection/train_images'
TEST_DIR = '../input/aptos2019-blindness-detection/test_images'
BATCH_SIZE = 64
#seed = 10
im_shape = (512,512)


#Datasets path
df_train_path = '../input/aptos2019-blindness-detection/train.csv'
df_test_path = '../input/aptos2019-blindness-detection/test.csv'

df_train = pd.read_csv(df_train_path)
df_test = pd.read_csv(df_test_path)

In [ ]:
# Reference: https://www.kaggle.com/shubhamgajbhiye/aptos-blindness-detection-eda-and-keras-resnet50
sns.set_style("white")
count = 1
plt.figure(figsize=[20, 20])
for img_name in df_train['id_code'][:15]:
    img = cv2.imread("../input/aptos2019-blindness-detection/train_images/%s.png" % img_name)[...,[2, 1, 0]]
    plt.subplot(5, 5, count)
    plt.imshow(img)
    plt.title("Image %s" % count)
    count += 1
    
plt.show()

## Initial View

In [ ]:
df_train.head()

In [ ]:
df_train.info()

Change type of `id_code` and `diagnosis` to string

In [ ]:
df_train['diagnosis'] = df_train['diagnosis'].astype(str)
df_train['id_code'] = df_train['id_code'].astype(str)+'.png'

df_test['id_code'] = df_test['id_code'].astype(str)+'.png'

# Data Augumentation 

In [ ]:
# With augmentation
data_generator = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        #rotation_range=20,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #preprocessing_function=preprocess_input,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        #fill_mode='nearest'
)
val_data_generator = ImageDataGenerator(
    #preprocessing_function=preprocess_input,
    validation_split=0.2,
    rescale=1./255
)

Reference: https://colab.research.google.com/github/drprajapati/APTOS-2019-Blindness-Detection/blob/master/APTOS_Blindness_Detection_Preprocessing.ipynb#scrollTo=siy27T6ig6q4

In [ ]:
# Generator para parte train
train_generator = data_generator.flow_from_dataframe(directory=TRAINING_DIR, target_size=im_shape, shuffle=True,
                                                     class_mode='categorical', batch_size=BATCH_SIZE,
                                                     dataframe=df_train, x_col='id_code', y_col='diagnosis'
                                                    )
# Generator para parte validação
validation_generator = val_data_generator.flow_from_dataframe(directory=TRAINING_DIR, target_size=im_shape, shuffle=False,
                                                              class_mode='categorical', batch_size=BATCH_SIZE, 
                                                              dataframe=df_train, x_col='id_code', y_col='diagnosis'
                                                             )

# Generator para dataset de teste
test_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_generator.flow_from_dataframe(directory=TEST_DIR, target_size=im_shape, shuffle=False,
                                                    class_mode=None, batch_size=BATCH_SIZE,
                                                    dataframe=df_test, x_col='id_code'#, y_col='diagnosis'
                                                   )

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

In [ ]:
# Visualizando alguns exemplos do dataset por meio do Generator criado
plt.figure(figsize=(15,15))
for i in range(9):
    #gera subfigures
    plt.subplot(330 + 1 + i)
    batch = train_generator.next()[0]*255
    image = batch[0].astype('uint8')
    plt.imshow(image)
plt.show()

# Create Simple Model

In [ ]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(im_shape[0],im_shape[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(10, kernel_size=(3,3), activation='selu'))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compila o modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
epochs = 6

#Callback to save the best model
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,verbose=1)
]

#Training
history = model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // BATCH_SIZE,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // BATCH_SIZE)

In [ ]:
def plot_curves(history_param):
    # Training curves
    history_dict = history_param.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']

    epochs_x = range(1, len(loss_values) + 1)
    plt.figure(figsize=(10,10))
    plt.subplot(2,1,1)
    plt.plot(epochs_x, loss_values, 'bo', label='Training loss')
    plt.plot(epochs_x, val_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation Loss and Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(2,1,2)
    acc_values = history_dict['accuracy']
    val_acc_values = history_dict['val_accuracy']
    plt.plot(epochs_x, acc_values, 'bo', label='Training acc')
    plt.plot(epochs_x, val_acc_values, 'b', label='Validation acc')
    #plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    plt.show()
    



In [ ]:
plot_curves(history)

In [ ]:
def view_result():
    # Load the best saved model
    model = load_model('model.h5')

    # Using the validation dataset
    score = model.evaluate(validation_generator)
    print('Val loss:', score[0])
    print('Val accuracy:', score[1])

    # Using the test dataset
    score = model.evaluate(test_generator)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    


In [ ]:
view_result()

# Transfer Learning from a Deep Model

In [ ]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))

x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=(3,3), activation='selu')(x)
x = Flatten()(x)
x = Dense(70, activation='selu')(x)
predictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing pretrained layers
for layer in base_model.layers:
    layer.trainable=False
    
optimizer = Adam()
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs = 10

# Saving the best model
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,verbose=1)
]

history = model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // BATCH_SIZE,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // BATCH_SIZE)

In [ ]:
plot_curves(history)

In [ ]:
view_result()